In [1]:
%reset -f

In [2]:
import pandas as pd
from sssom.parsers import parse_sssom_table
from sssom import compare_dataframes
from sssom.parsers import split_dataframe
from sssom.util import MappingSetDataFrame
from os.path import join
from oaklib import OntologyResource
from oaklib.implementations import SqlImplementation
import textdistance

In [3]:
lexmatch_file = "../mappings/mondo-sources-all-lexical.sssom.tsv"
mondo_sssom = "../ontology/tmp/mondo.sssom.tsv"
dir_name = "dataframes"

In [4]:
# Functions

def add_distance(df, col_name, txt_dist_pkg):
    df\
    .insert(\
            len(df.columns),\
            col_name,\
            df.apply\
            (\
                lambda x: txt_dist_pkg\
                 (\
                    x.subject_label.lower(), x.object_label.lower()\
                    if pd.notnull(x.object_label) else "99"\
                 ),\
                axis=1,\
            )\
           )

In [5]:
%%time
msdf_lex = parse_sssom_table(lexmatch_file)
msdf_mondo = parse_sssom_table(mondo_sssom)

# Use OAK to get 'object_label'
ontology_resource = OntologyResource(slug='../ontology/tmp/merged.db', local=True)
oi = SqlImplementation(ontology_resource)

CPU times: user 22.9 s, sys: 321 ms, total: 23.3 s
Wall time: 25 s


In [6]:
# msdf_mondo.df[msdf_mondo.df['object_id'].str.contains('ICD')]
"ICD10CM", "MONDO"
condition_1 = msdf_mondo.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_mondo.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_mondo.df['object_id'].str.contains("MONDO")
condition_4 = msdf_mondo.df['subject_id'].str.contains("ICD10CM")
msdf_mondo.df = msdf_mondo.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_mondo.df['object_label'] = msdf_mondo.df['object_id'].apply(lambda x: oi.label(x))
msdf_mondo.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label
51,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Unspecified,Precocious puberty
111,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.3,Unspecified,Discordant ventriculoarterial connection
113,MONDO:0000153,transposition of the great arteries,skos:broadMatch,ICD10CM:Q20.5,Unspecified,Discordant atrioventricular connection
202,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Unspecified,Disorders of mineral metabolism
222,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Unspecified,Erysipeloid


In [7]:
# msdf_lex.df[msdf_lex.df['object_id'].str.contains('ICD')]
condition_1 = msdf_lex.df['object_id'].str.contains("ICD10CM")
condition_2 = msdf_lex.df['subject_id'].str.contains("MONDO")
condition_3 = msdf_lex.df['object_id'].str.contains("MONDO")
condition_4 = msdf_lex.df['subject_id'].str.contains("ICD10CM")
msdf_lex.df = msdf_lex.df[(condition_1 & condition_2) | (condition_3 & condition_4)]
msdf_lex.df


,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string
28311,ICD10CM:A07.3,Isosporiasis,skos:exactMatch,MONDO:0018769,isosporiasis,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,Isosporosis
28326,ICD10CM:A15.0,Tuberculosis of lung,skos:closeMatch,MONDO:0021953,tuberculous fibrosis of lung,semapv:LexicalMatching,oaklib,0.500000,oio:hasExactSynonym,oio:hasRelatedSynonym,Tuberculous fibrosis of lung
28346,ICD10CM:A19,Miliary tuberculosis,skos:exactMatch,MONDO:0005848,miliary tuberculosis,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,disseminated tuberculosis
28347,ICD10CM:A19,Miliary tuberculosis,skos:exactMatch,MONDO:0005848,miliary tuberculosis,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,generalized tuberculosis
28363,ICD10CM:A21,Tularemia,skos:closeMatch,MONDO:0018077,tularemia,semapv:LexicalMatching,oaklib,0.500000,oio:hasExactSynonym,oio:hasRelatedSynonym,rabbit fever
...,...,...,...,...,...,...,...,...,...,...,...
36012,ICD10CM:T70.29,Other effects of high altitude,skos:closeMatch,MONDO:0021811,acute mountain sickness,semapv:LexicalMatching,oaklib,0.500000,oio:hasExactSynonym,oio:hasRelatedSynonym,Mountain sickness
36014,ICD10CM:T70.3,Caisson disease [decompression sickness],skos:closeMatch,MONDO:0020797,decompression sickness,semapv:LexicalMatching,oaklib,0.500000,oio:hasExactSynonym,oio:hasRelatedSynonym,Compressed-air disease
36033,ICD10CM:T78.3,Angioneurotic edema,skos:exactMatch,MONDO:0010481,angioedema,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,Quincke's edema
36041,ICD10CM:T79.6,Traumatic ischemia of muscle,skos:exactMatch,MONDO:0003528,Volkmann contracture,semapv:LexicalMatching,oaklib,0.800000,oio:hasExactSynonym,oio:hasExactSynonym,Volkmann's ischemic contracture


In [8]:
comparison_ms_diff = compare_dataframes(msdf_mondo.df, msdf_lex.df)
comparison_df = comparison_ms_diff.combined_dataframe
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_1', "MONDO_MAPPINGS")
comparison_df['comment'] = comparison_df['comment'].str.replace('UNIQUE_2', "LEXMATCH")
comparison_df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
50324,MONDO:0015123,obsolete rare inherited dyslipidemia,skos:broadMatch,ICD10CM:E78.0,Unspecified,Pure hypercholesterolemia,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
7249,MONDO:0002660,blepharochalasis,skos:exactMatch,ICD10CM:H02.3,Unspecified,Blepharochalasis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
24286,MONDO:0007950,mastocytosis,skos:broadMatch,ICD10CM:D47.0,Unspecified,Mast cell neoplasms of uncertain behavior,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
23167,MONDO:0007656,Gerstmann-Straussler-Scheinker syndrome,skos:exactMatch,ICD10CM:A81.82,Unspecified,Gerstmann-StrÃ¤ussler-Scheinker syndrome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
61163,MONDO:0019173,rabies,skos:broadMatch,ICD10CM:A82.0,Unspecified,Sylvatic rabies,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [9]:
mappings = ["LEXMATCH", "MONDO_MAPPINGS"]
unmapped_df = comparison_df[
    (comparison_df['comment'].str.contains("|".join(mappings)))
]

unmapped_df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
50324,MONDO:0015123,obsolete rare inherited dyslipidemia,skos:broadMatch,ICD10CM:E78.0,Unspecified,Pure hypercholesterolemia,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
7249,MONDO:0002660,blepharochalasis,skos:exactMatch,ICD10CM:H02.3,Unspecified,Blepharochalasis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
24286,MONDO:0007950,mastocytosis,skos:broadMatch,ICD10CM:D47.0,Unspecified,Mast cell neoplasms of uncertain behavior,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
23167,MONDO:0007656,Gerstmann-Straussler-Scheinker syndrome,skos:exactMatch,ICD10CM:A81.82,Unspecified,Gerstmann-StrÃ¤ussler-Scheinker syndrome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
61163,MONDO:0019173,rabies,skos:broadMatch,ICD10CM:A82.0,Unspecified,Sylvatic rabies,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [10]:
# Let reviewers check if this makes sense or no.
unmapped_lex_exact = unmapped_df[(unmapped_df['comment'] == 'LEXMATCH') & (unmapped_df['predicate_id'] == 'skos:exactMatch')]
unmapped_lex_exact.to_csv(join(dir_name, "unmapped_lex_exact.tsv"), sep='\t', index = False)
unmapped_lex_exact.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
29488,ICD10CM:D21,Other benign neoplasms of connective and other...,skos:exactMatch,MONDO:0024470,semapv:LexicalMatching,benign chondrogenic neoplasm,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,benign neoplasm of cartilage
30535,ICD10CM:G46.3,Brain stem stroke syndrome,skos:exactMatch,MONDO:0006827,semapv:LexicalMatching,lateral medullary syndrome,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,Wallenberg syndrome
29845,ICD10CM:E34.51,Complete androgen insensitivity syndrome,skos:exactMatch,MONDO:0019154,semapv:LexicalMatching,androgen insensitivity syndrome,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,Goldberg-Maxwell syndrome
29414,ICD10CM:C88.3,Immunoproliferative small intestinal disease,skos:exactMatch,MONDO:0015045,semapv:LexicalMatching,alpha-heavy chain disease,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,Alpha heavy chain disease
29415,ICD10CM:C88.3,Immunoproliferative small intestinal disease,skos:exactMatch,MONDO:0015045,semapv:LexicalMatching,alpha-heavy chain disease,LEXMATCH,oaklib,0.8,oio:hasExactSynonym,oio:hasExactSynonym,Mediterranean lymphoma


In [17]:
# Inspect why these are missing from SSSOM mappings
unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))
unmapped_mondo_exact = unmapped_df[(unmapped_df['comment'] == 'MONDO_MAPPINGS')& (unmapped_df['predicate_id'] == 'skos:exactMatch')]
# Add distances
# Add Levenshtein distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "levenshtein_dist", textdistance.levenshtein.distance)
# Add Jaccard Index [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "jaccard_index", textdistance.jaccard.distance)
# Add Monge-Elkan Distance [ 0: Perfect match]
add_distance(unmapped_mondo_exact, "monge_elkan", textdistance.monge_elkan.distance)
unmapped_mondo_exact.to_csv(join(dir_name, "unmapped_mondo_exact.tsv"), sep='\t', index = False)
unmapped_mondo_exact.head()


/var/folders/j5/bzs7jjf12j998fm88l0ssg740000gp/T/ipykernel_16730/4179603520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmapped_mondo_exact['object_label'] = unmapped_mondo_exact['object_id'].apply(lambda x: oi.label(x))


,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string,levenshtein_dist,jaccard_index,monge_elkan
7249,MONDO:0002660,blepharochalasis,skos:exactMatch,ICD10CM:H02.3,Unspecified,Blepharochalasis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
23167,MONDO:0007656,Gerstmann-Straussler-Scheinker syndrome,skos:exactMatch,ICD10CM:A81.82,Unspecified,Gerstmann-StrÃ¤ussler-Scheinker syndrome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,2,0.073171,1.974359
4536,MONDO:0001815,extrapyramidal and movement disease,skos:exactMatch,ICD10CM:G20-G26,Unspecified,Extrapyramidal and movement disorders (G20-G26),MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,14,0.326531,1.971429
4243,MONDO:0001724,supraglottis cancer,skos:exactMatch,ICD10CM:C32.1,Unspecified,Malignant neoplasm of supraglottis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,29,0.567568,1.952909
14029,MONDO:0004937,hypervitaminosis D,skos:exactMatch,ICD10CM:E67.3,Unspecified,Hypervitaminosis D,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000


In [12]:
combined_msdf = MappingSetDataFrame(df=comparison_df, prefix_map=msdf_lex.prefix_map, metadata=msdf_lex.metadata)
combined_msdf.df.head()

,subject_id,subject_label,predicate_id,object_id,mapping_justification,object_label,comment,mapping_tool,confidence,subject_match_field,object_match_field,match_string
50324,MONDO:0015123,obsolete rare inherited dyslipidemia,skos:broadMatch,ICD10CM:E78.0,Unspecified,Pure hypercholesterolemia,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
7249,MONDO:0002660,blepharochalasis,skos:exactMatch,ICD10CM:H02.3,Unspecified,Blepharochalasis,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
24286,MONDO:0007950,mastocytosis,skos:broadMatch,ICD10CM:D47.0,Unspecified,Mast cell neoplasms of uncertain behavior,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
23167,MONDO:0007656,Gerstmann-Straussler-Scheinker syndrome,skos:exactMatch,ICD10CM:A81.82,Unspecified,Gerstmann-StrÃ¤ussler-Scheinker syndrome,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN
61163,MONDO:0019173,rabies,skos:broadMatch,ICD10CM:A82.0,Unspecified,Sylvatic rabies,MONDO_MAPPINGS,NaN,NaN,NaN,NaN,NaN


In [13]:
%%time
df_dict = split_dataframe(combined_msdf)

CPU times: user 1.16 s, sys: 22 ms, total: 1.19 s
Wall time: 3.79 s


In [14]:
mondo_icd_list = [x for x in list(df_dict.keys()) if 'mondo' in x and "icd10" in x]
mondo_icd_list

['mondo_exactmatch_icd10cm',
 'mondo_broadmatch_icd10cm',
 'icd10cm_exactmatch_mondo',
 'icd10cm_narrowmatch_mondo',
 'icd10cm_closematch_mondo',
 'icd10cm_broadmatch_mondo']

In [15]:
for match in mondo_icd_list:
    fn = match + ".tsv"
    df_dict[match].df.to_csv(join(dir_name,fn), sep='\t', index = False)

In [16]:
df_dict['mondo_exactmatch_icd10cm'].df

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,comment
0,MONDO:0000088,precocious puberty,skos:exactMatch,ICD10CM:E30.1,Precocious puberty,Unspecified,MONDO_MAPPINGS
1,MONDO:0000226,mineral metabolism disease,skos:exactMatch,ICD10CM:E83,Disorders of mineral metabolism,Unspecified,MONDO_MAPPINGS
2,MONDO:0000237,erysipeloid,skos:exactMatch,ICD10CM:A26,Erysipeloid,Unspecified,MONDO_MAPPINGS
3,MONDO:0000245,tinea imbricata,skos:exactMatch,ICD10CM:B35.5,Tinea imbricata,Unspecified,MONDO_MAPPINGS
4,MONDO:0000261,adenoiditis,skos:exactMatch,ICD10CM:J35.02,Chronic adenoiditis,Unspecified,MONDO_MAPPINGS
...,...,...,...,...,...,...,...
1130,MONDO:0100347,carcinoid syndrome,skos:exactMatch,ICD10CM:E34.0,Carcinoid syndrome,Unspecified,MONDO_MAPPINGS
1131,MONDO:0100471,vitamin D deficiency,skos:exactMatch,ICD10CM:E55,Vitamin D deficiency,Unspecified,MONDO_MAPPINGS
1132,MONDO:0100491,generalized pustular psoriasis,skos:exactMatch,ICD10CM:L40.1,Generalized pustular psoriasis,Unspecified,MONDO_MAPPINGS
1133,MONDO:0800029,interstitial lung disease 2,skos:exactMatch,ICD10CM:J84.112,Idiopathic pulmonary fibrosis,Unspecified,MONDO_MAPPINGS
